In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow
import numpy as np
import cv2

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def img2np(path,resize=None):
    '''
    이미지의 주소를 입력받아 이미지를 불러온 후, numpy로 변경해주는 함수이다.
    '''
    img = cv2.imread(path)
    
    if resize:
        img = cv2.resize(img,resize)

    imgArray = np.array(img)
    
    return imgArray

In [ ]:
def grayscale(img):
    #ITU-R BT.601에서 정의된 식을 사용하여 기존 rgb이미지를 grayscale로 변경한다.
    r,g,b = img[:,:,0],img[:,:,1],img[:,:,2]
    return 0.2989 * r + 0.5870 * g + 0.1140 * b

In [ ]:
def dsi(imL,imR):
    # 두 이미지의 dsi를 구한다.
    # 1. 이미지의 H와 W를 구한다.
    h,w = imL.shape
    # 2. 두 이미지의 DSI를 저장할 텐서를 선언한다. 이 때 사이즈는 (HxWxW)이다.
    #    기본값을 500으로 설정했습니다. 효율성을 위해 dsi 전체가 아닌 필요한 부분만을 계산하기로 했다.
    #    만약 기본값이 0이라고 한다면, 후에 costMap에서 계산하지 않은 부분의 dsi값이 낮아 제대로된 costMap을 구할 수 없어, 이를 해결하기 위해 높은 값으로 500을 설정해주었다.
    dsi = np.ones((h,w,w))* 500
    
    for row in range(h):
        for r in range(w):
            for c in range(w):
                if r- c >= w*0 or r - c < -w*0.2:
                    #이때, 각 dsi의 x축이 right image, y축이 left image이기 때문에 right image point - left image point > 0인 부분만 계산할 수 있도록 했다.
                    #또한. right image point - left image point <0인 부분 모두를 계산하지 않고, 더욱 효율적인 계산을 하기 위해 휴리스틱한 방법으로 적절한 범위를 찾았고,
                    #그 결과, right image point - left image point < - image width * 0.2까지 이미지의 차이가 있지 않아 이 값으로 설정해주었다.
                    #                    right image
                    #                   ________
                    #         left     |
                    #         image    |
                    continue
                left = imL[row,r]
                right= imR[row,c]
                # dsi 값으로 두 값의 차의 제곱으로 설정해주었다.
                dsi[row,r,c]= ((left-right)**2)/32
        if row % 50 == 0:
            print(f"-----row{row}-------")
            print("DSI Left")
            cv2_imshow(dsi[row])
    return dsi

In [ ]:
def costMap(dsi):
    # dsi 텐서를 받아 costMap을 생성해 반환해줍니다.
    h,w = dsi.shape[:2]
    # 휴리스틱한 방법을 사용해 occ의 적절한 값을 찾았다.
    occ = 3
    # 각 dsi에 대한 costMap을 저장하는 텐서를 생성해준다.
    cost = np.zeros((h,w,w))
    for row in range(h):
        for r in range(w):
            for c in range(w):
                # 기본적으로 각 costMap의 0번째 열과 0번째 행에는 occlusion 값들을 설정해준다.
                if r == 0:
                    cost[row,r,c]=occ*c
                elif c == 0:
                    cost[row,r,c]=occ*r
    # dynamicMap 생성
    for row in range(h):
        for r in range(w):
            for c in range(w):
                # 대각선 / 왼->오 / 위-> 아래 방향중 가장 낮은 값을 현재 위치의 cost로 설정해준다.
                m1 = cost[row,r-1,c-1]+dsi[row,r,c]
                m2 = cost[row,r-1,c]+occ #오른쪽으로 감
                m3 = cost[row,r,c-1]+occ #아래로 내려감
                move = min(m1,m2,m3)
                cost[row,r,c] = move
        if row % 50 == 0:
            print(f"----{row}----")
            cv2_imshow(cost[row]-np.mean(cost[row]))
    return cost

In [ ]:
# costMap(dynamicMap)을 통해 최적의 route를 구한다.
# 여기서 생기는 route는 결국 같은 row에서 route(a,b)에서 a와 b가 가장 같아요! 란 의미야
# 그럼
def findRoute(cost):
    # CostMap을 받아
    h,w = cost.shape[:2]
    # costMap을 통해서 각 row에 대한 depth값을 저장할 배열을 생성합니다.
    # route_right: 오른쪽 이미지 기준의 depth값을 저장한다.
    # route_left: 왼쪽 이미지 기준의 depth값을 저장한다.
    # route_right = np.zeros((h,w))
    # route_left = np.zeros((h,w))
    # 최종 depth image가 될 2차원 배열을 생성해준다.
    depth_right = np.zeros((h,w))
    depth_left = np.zeros((h,w))
    for row in range(h):
        # 마지막 인덱스(w-1,w-1)부터 역으로 optimal path를 찾아, depth 값을 저장한다.
        r,c = w-1,w-1
        while r != -1 and c != -1:
            # m1 대각선 m2 가로로(왼쪽) m3 세로로(위) 이동한다는 의미이다.
            # 위 세 값중 가장 낮은 값이 optimal path이다.
            m1 = cost[row,r-1,c-1]
            m2 = cost[row,r,c-1]
            m3 = cost[row,r-1,c]
            move = min(m1,m2,m3)
            
            # depth 값들을 업데이트 해준다.
            # 오른쪽 이미지를 기준으로 하는 depth image는 column, 왼쪽 이미지를 기준으로하는 depth image는 row를 기준으로 업데이트를 해준다.
            depth_right[row,c] = abs(r-c)*4
            depth_left[row,r]=abs(r-c)*4
            # 이동 방향으로 r,c를 업데이트 해준다.
            if m1 == move:
                r,c = r-1,c-1
            elif m2 == move:
                r,c = r,c-1
            else:
                r,c = r-1,c
        # print(row)
    print("왼쪽 이미지 기준 depth map")
    cv2_imshow(depth_left)
    print("오른쪽 이미지 기준 depth map")
    cv2_imshow(depth_right)
    return depth_left,depth_right

In [ ]:
if __name__ =='__main__':
    imR = img2np(path = '/content/drive/MyDrive/Colab Notebooks/cv/images/im2_2.png',resize=(224,224))
    imL = img2np(path = '/content/drive/MyDrive/Colab Notebooks/cv/images/im6_2.png',resize=(224,224))
    imL=grayscale(imL)
    imR=grayscale(imR)

    cv2_imshow(imL)
    cv2_imshow(imR)
    print(f"=====DSI=====")
    mydsi = dsi(imL,imR)
    print(f"=====CostMap=====")
    mycost= costMap(mydsi)
    depth_left,depth_right=findRoute(mycost)

In [ ]:
# 제안하는 함수들

# 새롭게 제안한 depth 공식이다.(실험 1)
def findRoute_propose(cost):
    h,w = cost.shape[:2]
    depth_right = np.zeros((h,w))
    depth_left = np.zeros((h,w))
    for row in range(h):
        r,c = w-1,w-1
        while r != -1 and c != -1:
            m1 = cost[row,r-1,c-1]
            m2 = cost[row,r,c-1]
            m3 = cost[row,r-1,c]
            move = min(m1,m2,m3)
            p = 0.5
            # abs와 제곱을 선형결합한다.
            depth_right[row,c] = (((r-c)**2)/4) *(1-p) + abs(r-c)*4 *p
            depth_left[row,r]=(((r-c)**2)/4) *(1-p) + abs(r-c)*4 *p
            if m1 == move:
                r,c = r-1,c-1
            elif m2 == move:
                r,c = r,c-1
            else:
                r,c = r-1,c
        # print(row)
    print("왼쪽 이미지 기준 depth map")
    cv2_imshow(depth_left)
    print("오른쪽 이미지 기준 depth map")
    cv2_imshow(depth_right)
    return depth_left,depth_right

In [ ]:
import copy
# 새롭게 제안하는 depth map 후처리 방법이다. (실험 2)
def depth_relu(depth,hold):
    depth = copy.deepcopy(depth)
    h,w = depth.shape
    for r in range(h):
        for c in range(w):
            if depth[r,c] > hold:
                depth[r,c] = hold
    print(hold)
    cv2_imshow(depth)

In [ ]:
# 새롭게 제안하는 depth map 후처리 방법이다. (실험 3)

def depth_LeakyReLU(depth,p):
    depth = copy.deepcopy(depth)
    h,w = depth.shape

    sorted_depth = depth.reshape(-1,1)
    sorted_depth = np.squeeze(depth)
    sorted_depth = np.sort(sorted_depth)
    hold = sorted_depth[0,int(len(sorted_depth)*p)]
    print(hold)
    for r in range(h):
        for c in range(w):
            if depth[r,c] > hold:
                depth[r,c] = hold*0.5 + depth[r,c]*0.5
    print(p,hold)
    cv2_imshow(depth)